# Data Science - Social Media Analytics SoSe 24 📊🔍

## Problemset 1 📝

This notebook represents my submission for the weekly tasks in Social Media Analytics for the summer semester of 2024.

### Authors 👥
- **Martin Brucker** (942815) 🧑‍💻
- **Frederik Brinkmann** (943915) 🧑‍💻

**Due**: Friday, 5th April 2024, 11:59 PM

**Contact Information**: martin.brucker@student.fh-kiel.de 📧


# Exersice 1 🤠

Your task is to scrape articles from Deutsche Welle (DW) along with additional metadata about
the articles.

Use the Python Package BeautifulSoup to extract the following information from this
website and store it in a Python dictionary:

1. author
1. date
1. title
1. summary (text printed in bold letters at the beginning)
1. main text
1. related topics (Minorities, Women’s rights, . . . )


• Analyze the article: what additional information (metadata or other) related to this article
is available? Make a list of such items. Then pick one of these items, extract it also from
this website, and update your dictionary with this item.


• Print the dictionary

In [2]:
# importing the required lib's
import requests
import pandas as pd

# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [3]:
## creating an instance of the BeautifulSoup class and setup the html parser

# the URL is the given URL from wich the html code is downloaded
url = "https://www.dw.com/en/why-south-korean-women-arent-having-babies/a-68419317"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

In [29]:
# Extracting information directly without creating separate variables
resultDict = {
    "author": soup.find("div", class_="author-details").get_text(),
    "date": soup.time.get_text(),
    "title": soup.title.get_text(),
    "summary": soup.find(class_="teaser-text").get_text(),
    "main_text": [p.get_text() for p in soup.find_all("p") 
                  if "cookie__text" not in p.get("class", []) and "vjs-no-js" not in p.get("class", [])],
    "related_topics": [a.get_text() for a in soup.aside.find_all("a")]
}

# Note: The 'main_text' filters out unnecessary <p> elements
# with classes 'cookie__text' and 'vjs-no-js'

#### Brainstorming additional information

* Image information (amount/url/size/alt text)
* Subsection Headlines (amount/text)
* Links to other pages (amount/connections)


In [43]:
## getting the sub-section headlines from the article

# add the sub_sections to the earlyer created dictionary
resultDict["sub_headlines"] = [h2.get_text() for h2 in soup.find_all("h2", class_=None)]


In [44]:
# printing the final dict
resultDict

{'author': 'Julian Ryall',
 'date': '03/01/2024',
 'title': "Why South Korean women aren't having babies – DW – 03/01/2024",
 'summary': 'New statistics show a record low number of children were born last year in South Korea, with women citing a desire for a career and to push back against a male-dominated society as key reasons.',
 'main_text': ['New statistics show a record low number of children were born last year in South Korea, with women citing a desire for a career and to push back against a male-dominated society as key reasons.',
  'When she was younger, Hyobin Lee yearned to be a mother. There came a point, however, when she had to make a difficult decision. Ultimately, she chose her career over a family and is now a successful academic in the South Korean city of Daejeon.',
  "Lee, now 44, is just one of millions of Korean women who are making a conscious decision to remain childless — resulting in the nation's fertility rate dropping to a new record low.",
  'The fertility

# Exersice 2 🤠

- Your task is to scrape all articles (only media type “Article”) from 01.03.2024 extract-
ing the same type of information as in exercise 1. Store the information in a Pandas
DataFrame such that a single row represents one article, and that the columns represent
the extracted features (author, date, . . . ). Hint: First, you need to extract the urls to
the single articles published on that day. Secondly, you need to open, parse, and extract
information from each of these articles.

- Scrape politly by including delays between requests and by scraping not more data than
you actually need.
- Print the shape of the DataFrame and display its first five rows. Display it in such a way
that all columns are visible.

In [46]:
# importing more libs to transform into datetime and set a sleep timer
import datetime
import time

#### Task Split: Collecting URLs and Extracting Relevant Data

The task will be divided into two major code blocks:

1. **URL Collection (01.03.2024)**:
   In this block, we'll collect all URLs to articles published on March 1, 2024.

2. **Data Extraction from Links**:
   In the second code block, we'll extract relevant data such as author names, publication dates, and article titles from the collected URLs.


Note: for sake of scraping we are not going to use the &from=02.04.2024 from the url 


In [57]:
articleLinks = []
page = 10
more_pages = True
target_date = "01.03.2024"

while more_pages == True:

    print(f"Now scraping page {page}")

    url = f"https://www.dw.com/search/?languageCode=en&contentType=ARTICLE&from={target_date}&to={target_date}&sort=DATE&resultsCounter={page}"
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    
    news = soup.select("div.searchResult")
    articleLinks.extend([article.a["href"] for article in news])

    if soup.select("a.addPage"):
        page += 10
    else:
        more_pages = False


    time.sleep(1)   

print(f"Finished scraping {len(articleLinks)} article URL's")

Now scraping page 10
Now scraping page 20
Now scraping page 30
Now scraping page 40
Finished scraping 92 article URL's


In [52]:
i = 0
result = []

for article in articleLinks:
    if(i % 10 == 0):
        print(f"Finished {i/len(articleLinks)} %")

    url = f"https://www.dw.com{article}"
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")

    data = {}

    # sadly the authoer is not so easy to extract so in the following tree propular examples of author formating is used to get the author
    if soup.find_all("div", class_="author-details"):
        data["author"] = soup.find_all("div", class_="author-details")
    elif soup.find_all("span", class_="author"):
        data["author"] = soup.find_all("span", class_="author")
    else:
        data["author"] = soup.find_all("div", class_="author")
    
    try: 
        data["date"] = soup.time.getText()
    except: 
        print("noDate")

    try:
        data["title"] = soup.title.getText()
    except:
        print("no title")

    try:
        data["summary"] = soup.find_all(class_="teaser-text")[0].getText()
    except:
        print("no summary")

    try:
        data["text"] = [p.getText() for p in soup.find_all("p", class_=lambda x: x not in ["cookie__text", "vjs-no-js"])]
    except:
        print("no text")
    
    data["related_topics"] = soup.aside

    result.append(data)
    i += 1

Finished 0.0 %
noDate
noDate
Finished 0.10869565217391304 %
noDate
noDate
Finished 0.21739130434782608 %
Finished 0.32608695652173914 %
noDate
noDate
Finished 0.43478260869565216 %
Finished 0.5434782608695652 %
noDate
no summary
Finished 0.6521739130434783 %
noDate
noDate
Finished 0.7608695652173914 %
Finished 0.8695652173913043 %
noDate
no summary
Finished 0.9782608695652174 %


In [60]:
import requests
from bs4 import BeautifulSoup

def scrape_article_data(article_links):
    result = []

    for i, article in enumerate(article_links):
        if i % 10 == 0:
            print(f"Finished {i / len(article_links) * 100:.2f}%")

        url = f"https://www.dw.com{article}"
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")

        data = {}
        
        try:
            data["author"] = soup.find("div", class_="author-details").get_text()
        except AttributeError:
            data["author"] = "Author not found"

        try:
            data["date"] = soup.time.get_text()
        except AttributeError:
            data["date"] = "Date not found"

        try:
            data["title"] = soup.title.get_text()
        except AttributeError:
            data["title"] = "Title not found"

        try:
            data["summary"] = soup.find(class_="teaser-text").get_text()
        except AttributeError:
            data["summary"] = "Summary not found"

        data["main_text"] = [p.get_text() for p in soup.find_all("p")
                             if "cookie__text" not in p.get("class", []) and "vjs-no-js" not in p.get("class", [])]

        data["related_topics"] = [a.get_text() for a in soup.aside.find_all("a")] if soup.aside else []

        result.append(data)

    return result

# Example usage:
scraped_data = scrape_article_data(articleLinks)

Finished 0.00%
Finished 10.87%
Finished 21.74%
Finished 32.61%
Finished 43.48%
Finished 54.35%
Finished 65.22%
Finished 76.09%
Finished 86.96%
Finished 97.83%
[{'author': 'Dirk Kaufmann', 'date': '03/01/2024', 'title': 'Apple pulls the plug on its self-driving e-car project – DW – 03/01/2024', 'summary': 'Apple has ended its decadelong autonomous vehicle effort, known as Project Titan. The stock market breathed a sigh of relief in response, with insiders pointing to areas where Apple should redouble its efforts instead.', 'main_text': ['Apple has ended its decadelong autonomous vehicle effort, known as Project Titan. The stock market breathed a sigh of relief in response, with insiders pointing to areas where Apple should redouble its efforts instead.', 'Traditional car manufacturers in Europe, Asia and the US face a number of problems, some of them of their own making. To make matters worse,\xa0the creeping concern that tech companies could crowd legacy firms out of the personal mobil

In [53]:
articlesAsDataFrame = pd.DataFrame(scraped_data)
articlesAsDataFrame.head()

,author,date,title,summary,text,related_topics
0,"[[[<span class=""sc-ivDvhZ iBEcrE sc-bYUneI clA...",03/01/2024,Apple pulls the plug on its self-driving e-car...,Apple has ended its decadelong autonomous vehi...,[Apple has ended its decadelong autonomous veh...,"[[Electric vehicles], [Apple], [Artificial int..."
1,[],03/01/2024,Iran elections: Low turnout in parliamentary v...,Iran's parliamentary elections saw low voter t...,[Iran's parliamentary elections saw low voter ...,"[[Iran protests], [Jina Mahsa Amini], [Iran]]"
2,[],03/01/2024,Germany: Tesla plant protesters to spend week ...,Treehouses and a piano concert in the woods: A...,[Treehouses and a piano concert in the woods: ...,"[[Rhine River], [Poverty in Germany], [Greta T..."
3,[],03/01/2024,Nicaragua says Germany facilitates genocide by...,Nicaragua has filed a case at the Internationa...,[Nicaragua has filed a case at the Internation...,"[[Rhine River], [Poverty in Germany], [Robert ..."
4,"[[[<span class=""sc-ivDvhZ iBEcrE sc-bYUneI clA...",03/01/2024,Why some African countries have strange shapes...,Arbitrary boundaries were drawn on maps to sep...,[Arbitrary boundaries were drawn on maps to se...,"[[Namibia], [Cameroon], [Togo], [Tanzania]]"


In [38]:
# transform date into datetime
articlesAsDataFrame["date"] = pd.to_datetime(articlesAsDataFrame["date"], format="%m/%d/%Y")

# transform related_topics into related_topics
# maybe formating it in multible columns

#check max related_topics length care of none
articlesAsDataFrame["related_topics"].apply(lambda x: len(x) if x else 0).max()

articlesAsDataFrame["related_topics"]

0                                          [[Lifestyle]]
1                                                   None
2      [[BRICS], [Hindu nationalism], [Manipur], [Nar...
3                                                   None
4                                                   None
                             ...                        
355    [[Black Sea], [BRICS], [Dmitry Medvedev], [Rus...
356    [[Black Sea], [Turkey elections], [Turkey-Syri...
357                     [[Israel], [Benjamin Netanyahu]]
358                     [[Israel], [Benjamin Netanyahu]]
359                                        [[The Hague]]
Name: related_topics, Length: 360, dtype: object

In [ ]:
# transforming the data usefull for further research

# transform date into datetime
# articlesAsDataFrame["date"] = pd.to_datetime(articlesAsDataFrame["date"])

# unwrap the outer array from the related_topics with the soup .getText(), check if none

# change dtype of the related_topics into categories
# articlesAsDataFrame["related_topics"] = articlesAsDataFrame["related_topics"].astype("category")

In [ ]:
articlesAsDataFrame["related_topics"]

0     [[Minorities], [Women's rights], [Taliban], [E...
1     [[Black Sea], [BRICS], [Dmitry Medvedev], [Rus...
2                                                  None
3     [[Artificial intelligence], [Asia], [South Kor...
4     [[EU migration policy], [BRICS], [Taiwan], [Uy...
5        [[Palestinian territories], [Israel], [Hamas]]
6                [[LGBTQ+ rights in Africa], [Namibia]]
7                 [[Michael Schumacher], [Formula One]]
8     [[BRICS], [Hindu nationalism], [Manipur], [Nar...
9                                [[Olympics], [France]]
10    [[Law and Justice], [Medical marijuana], [Rhin...
11    [[Medical marijuana], [Rhine River], [Poverty ...
12    [[Bayern Munich], [Borussia Dortmund], [Bundes...
13    [[BRICS], [China-Taiwan crisis], [Taiwan], [Uy...
14    [[Black Sea], [Turkey elections], [Turkey-Syri...
15    [[BRICS], [Hindu nationalism], [Manipur], [Nar...
16    [[Rhine River], [Poverty in Germany], [Robert ...
Name: related_topics, dtype: object